In [ ]:
import h5py
import matplotlib.pyplot as plt
import os
import numpy as np

images_dir = "./output/hdri_background/hdf5_format/"

In [ ]:
""" Option 1 - Shapes of the output """

for i in range(len(os.listdir(images_dir))):
    image = str(i) + ".hdf5"
    print("-"*5, image, "-"*5)
    image_dir = images_dir + image
    with h5py.File(image_dir, 'r') as file:
        def print_structure(name, obj):
            if isinstance(obj, h5py.Group):
                print(f"Group: {name}")
            elif isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name} - shape: {obj.shape}, dtype: {obj.dtype}")

        # Traverse and print the structure
        file.visititems(print_structure)
    print("\n")

In [ ]:
""" Option 2 - Looking at the contents of the output """

for i in range(len(os.listdir(images_dir))):
    image = str(i) + ".hdf5"
    print("-"*5, image, "-"*5)
    image_dir = images_dir + image
    with h5py.File(image_dir, 'r') as file:
        # View the blender_proc_version
        version = file['blender_proc_version'][()]
        print("BlenderProc Version:", version.decode('utf-8'))

        # View a small section of the colors dataset
        colors = file['colors'][:5, :5] # 5x5 sample
        print("Colors dataset sample (5x5 pixels):\n", colors)

        # # View a small section of the depth dataset
        # depth = file['depth'][:5, :5]  # 5x5 sample
        # print("Depth dataset sample (5x5 pixels):\n", depth)

        # # View a small section of the normals dataset
        # normals = file['normals'][:5, :5]
        # print("Normals dataset sample (5x5 pixels):\n", normals)

        segmaps = file['category_id_segmaps'][:5, :5]
        print("Instance Segmaps sample (5x5 pixels):\n", segmaps)
    
    print("\n")

In [ ]:
""" Option 3 - Plot images using matplotlib """

def plot_hdf5(file_name):
    file_path = images_dir + file_name
    # Load the HDF5 file
    with h5py.File(file_path, 'r') as file:
        colors = file['colors'][:]
        segmaps = file['category_id_segmaps'][:]

    # Plot the images side by side
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(file_name)

    # Display the color image
    axes[0].imshow(colors)
    axes[0].set_title("Colors")
    axes[0].axis("off")

    # Display the segmaps image
    axes[1].imshow(segmaps)
    axes[1].set_title("Category ID Segmaps")
    axes[1].axis("off")

    plt.show()


for i in range(len(os.listdir(images_dir))):
    image = str(i) + ".hdf5"
    plot_hdf5(image)

In [ ]:
""" Option 4 - The real content of an hdf5 files """

%run "-m" "blenderproc" "vis" "hdf5" "./output/hdri_background/hdf5_format/1508.hdf5"


In [ ]:
""" Option 5 - Save the hdf5 files as jpgs """

import matplotlib.cm as cm
from PIL import Image

jpg_format_dir = os.path.join(images_dir.split("hdf5")[0], "jpg_format")

for i in range(len(os.listdir(images_dir))):
    # Load the HDF5 file
    file = str(i) + ".hdf5"
    file_path = os.path.join(images_dir, file)
    # Create the directory that will contain the JPEG images, and set the file paths.
    os.makedirs(jpg_format_dir, exist_ok=True)
    colors_dir = os.path.join(jpg_format_dir, str(i) + "_color.jpg")
    segmaps_dir = os.path.join(jpg_format_dir, str(i) + "_segmaps.jpg")

    with h5py.File(file_path, 'r') as file:
        # Read the 'colors' dataset and save it as a JPEG image.
        colors = file['colors'][:]
        color_img = Image.fromarray(colors, 'RGB')
        color_img.save(colors_dir, "JPEG")
        
        # Read the 'category_id_segmaps' dataset.
        segmaps = file['category_id_segmaps'][:]
        # Normalize and map the segmaps to a color range for visibility.
        segmaps_normalized = (segmaps - segmaps.min()) / (segmaps.max() - segmaps.min())
        segmaps_colormap = (cm.viridis(segmaps_normalized)[:, :, :3] * 255).astype(np.uint8)
        # Save the segmaps as a JPEG image.
        segmaps_img = Image.fromarray(segmaps_colormap)
        segmaps_img.save(segmaps_dir, "JPEG")